In [ ]:
import pandas as pd
import glob
import os
import re
from collections import Counter
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from nltk.corpus import stopwords
import nltk
from wordcloud import WordCloud

# Download the stopwords if you haven't already
nltk.download('stopwords')

repo_owner = 'OpenBB-finance'
repo_name = 'OpenBB'

folder_name = f"{repo_owner}_{repo_name}"

list_of_files = glob.glob(os.path.join(folder_name, 'stargazer_info_*_processed.csv'))
latest_file = max(list_of_files, key=os.path.getctime) if list_of_files else None

# Read the CSV file
df = pd.read_csv(latest_file)

In [ ]:
# Replace 'United States' with 'USA' in the Country column
df['Country'] = df['Country'].replace('United States', 'USA')

# Replace variations regardless of case
replacements = {
    'Amazon Web Services': 'Amazon',
    'Aws': 'Amazon',
}
df['StandardCompany'] = df['StandardCompany'].replace(replacements)

replacements = {
    'Swe': 'Software Engineer',
    'Cs Student': 'Computer Science Student',
    'Cto': 'CTO',
    'Ceo': 'CEO',
    'Phd': 'PhD',
}
df['Job'] = df['Job'].apply(lambda x: x.lower().capitalize() if isinstance(x, str) else x)
df['Job'] = df['Job'].apply(lambda x: ' '.join(word.capitalize() for word in x.split()) if isinstance(x, str) else x)
df['Job'] = df['Job'].replace(replacements)

# Save the updated dataframe
df.to_csv(latest_file.replace('.csv', '_processed.csv'), index=False)

In [ ]:
df['Starred at'] = pd.to_datetime(df['Starred at'])
# Resample by week and count the occurrences
weekly_counts = df.resample('W-MON', on='Starred at').size()

fig = go.Figure()
fig.add_trace(go.Scatter(x=weekly_counts.index, y=weekly_counts.values, mode='lines'))
fig.update_layout(
    title='Weekly Stars Over Time',
    xaxis_title='Week Starting',
    yaxis_title='Number of Stars per Week',
    width=1000,
    height=400
)
fig.show()

In [ ]:
# Define country groupings
regions = {
    'North America': ['USA', 'Canada', 'United States of America'],
    
    'Europe': ['Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 
               'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Czechia', 'Denmark', 'Estonia',
               'Finland', 'France', 'Germany', 'Gibraltar', 'Greece', 'Hungary', 'Iceland',
               'Ireland', 'Isle of Man', 'Italy', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania',
               'Luxembourg', 'Malta', 'Moldova', 'Montenegro', 'Netherlands', 'North Macedonia',
               'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'Serbia', 'Slovakia',
               'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine', 'United Kingdom'],
    
    'Asia': ['Afghanistan', 'Armenia', 'Azerbaijan', 'Bangladesh', 'Bhutan', 'Brunei', 'Cambodia',
             'China', 'Georgia', 'Hong Kong', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel',
             'Japan', 'Jordan', 'Kazakhstan', 'Korea (Republic of)', 'Kuwait', 'Kyrgyzstan',
             'Lebanon', 'Macau', 'Malaysia', 'Mongolia', 'Myanmar', 'Nepal', 'Oman', 'Pakistan',
             'Palestine', 'Philippines', 'Qatar', 'Republic of Korea', 'Saudi Arabia', 'Singapore',
             'South Korea', 'Sri Lanka', 'Taiwan', 'Thailand', 'Turkey', 'Turkmenistan', 'Türkiye',
             'United Arab Emirates', 'Vietnam', 'Yemen'],
    
    'South America': ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Peru',
                     'Uruguay', 'Venezuela'],
    
    'Africa': ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Cameroon', 'Djibouti',
               'Egypt', 'Eswatini', 'Ethiopia', 'Ghana', 'Ivory Coast', "Côte d'Ivoire", 'Kenya',
               'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Morocco',
               'Mozambique', 'Namibia', 'Nigeria', 'Senegal', 'Somalia', 'South Africa', 'Sudan',
               'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zimbabwe'],
    
    'Oceania': ['Australia', 'Fiji', 'New Zealand', 'Papua New Guinea'],
    
    'Central America & Caribbean': ['Bahamas', 'Cayman Islands', 'Costa Rica', 'Cuba', 
                                  'Dominican Republic', 'El Salvador', 'Grenada', 'Guatemala',
                                  'Jamaica', 'Mexico', 'Panama', 'Puerto Rico', 'Saint Kitts and Nevis',
                                  'Trinidad and Tobago', 'Turks and Caicos Islands']
}

# Create a function to map countries to regions
def get_region(country):
    for region, countries in regions.items():
        if country in countries:
            return region
    return None

# Create region counts
region_counts = df['Country'].apply(get_region).value_counts()

# Create the plot
fig = go.Figure(data=[
    go.Bar(x=region_counts.index, y=region_counts.values)
])

fig.update_layout(
    title='Stargazers by Major Regions',
    xaxis_title='Region',
    yaxis_title='Number of Stargazers',
    width=1000,
    height=400
)

fig.update_xaxes(tickangle=0)  # No need for angled labels with fewer categories

fig.show()

In [ ]:
top_countries = df['Country'].value_counts().head(20)

fig = go.Figure(data=[
    go.Bar(x=top_countries.index, y=top_countries.values)
])

fig.update_layout(
    title='Top Countries of Stargazers',
    xaxis_title='Country',
    yaxis_title='Number of Stargazers',
    width=1000,
    height=400
)

fig.update_xaxes(tickangle=45)

fig.show()

In [ ]:
top_regions = df['Region'].value_counts().head(20)

fig = go.Figure(data=[
    go.Bar(x=top_regions.index, y=top_regions.values)
])

fig.update_layout(
    title='Top Region of Stargazers',
    xaxis_title='Region',
    yaxis_title='Number of Stargazers',
    width=1000,
    height=400
)

fig.update_xaxes(tickangle=45)

fig.show()

In [ ]:
top_company = df['StandardCompany'].value_counts().head(20)

fig = go.Figure(data=[
    go.Bar(x=top_company.index, y=top_company.values)
])

fig.update_layout(
    title='Top Company of Stargazers',
    xaxis_title='Company',
    yaxis_title='Number of Stargazers',
    width=1000,
    height=400
)

fig.update_xaxes(tickangle=45)

fig.show()

In [ ]:
top_job = df['Job'].value_counts().head(40)

fig = go.Figure(data=[
    go.Bar(x=top_job.index, y=top_job.values)
])

fig.update_layout(
    title='Top Job of Stargazers',
    xaxis_title='Job',
    yaxis_title='Number of Stargazers',
    width=1000,
    height=400
)

fig.update_xaxes(tickangle=45)

fig.show()

In [ ]:
job_categories = {
    'Engineering & Development': [
        'engineer', 'engineering', 'developer', 'programmer', 'coder', 'software',
        'software dev', 'swe', 'fullstack', 'full stack', 'backend', 'frontend',
        'web dev', 'mobile', 'ios', 'node', 'php', '.net', 'javascript', 'python',
        'infrastructure', 'infra', 'sde', 'dev', 'programming', 'hacker', 'it',
        'automation', 'maker', 'technical', 'full-stack', 'front-end', 
        'internet plumber', 'bug farmer', 'bug fixer',
        'web platform', 'senior staff se', 'working on cloud & virtualization',
        'dereferencing null pointers', 'net archeologist', 'tools', 'technologist',
        'coding', 'technology', 'cloud', 'tech', 'junior',
        'Analista Desenvolvedor', 'Analista Programador Computación', 'App Mechanic',
        'Bug Finder&fixer', 'Code', 'Code Artisan', 'Code Farmer', 'Code Monkey',
        'Code Player', 'Code Wrangler', 'Desenvolvedor Web', 'Dotnet Se', 
        'Embedded System Eng.', 'Eng', 'Engenheiro De Computação', 'Enginner',
        'Front End Beginner', 'Java Programer', 'Programação', 'Programer',
        'Web 3d Programer', '프로그래머', 'プログラマー', '程序员', '程序猿', '软件工程师', '开发者',
        'Cowboy Of The Front End', 'Creating React Native Apps', 
        'Dba', 'Mern-stack Maestro', 'Perekayasa Perangkat Lunak',
        'Power User/automator', 'Pro Copy Paster & Tab-enterer',
        'Pyspark Summoner/hadoop Zealot', 'Rust Tooling',
        'Script Kiddie', 'Script-kiddie', 'Slayer Of Bugs',
        'Spaghetti Chef', 'Systems/network/comms Hack',
        'Técnico En Informática', 'Webrtc Nerd',
        'Xslt Enthusiast & E-invoicing Expert',
        'Étudiante En Génie Logiciel', '微信小程序《我是面霸》作者', 'Ingeniero'
    ],
    
    'Data & Analytics': [
        'analyst', 'analytics', 'data scientist', 'data science', 'visualization',
        'charts', 'tables', 'bi ', 'business intelligence', 'data steward',
        'data specialist', 'quantitative', 'quant', 
        'data enthusiast', 'full-stack data enthusiast', 'data barista',
        'data processing', 'data', 'Cientista De Dados', 'Decision Science', 'Ds Magician',
        'Search & Discovery (relevancy & Personalization)',
        'Search Ranking', 'Convex Optimisation', 'Stat'
    ],
    
    'Research & Science': [
        'researcher', 'scientist', 'research', 'phd', 'ph.d', 'postdoc', 'post-doc',
        'professor', 'assistant professor', 'associate professor', 'fellow',
        'biologist', 'physicist', 'computational', 'biochemist', 'astronomer',
        'bioinformatics', 'research assistant', 'compsci', 'computer science', 'cs',
        'mathematician', 'statistician', 'teacher',
        'Bioinformatician', 'Biólogo', 'Clinical Psycologist', 'Computer Systems Nerd',
        'Comsci Undergrad', 'Ee Undergrad', 'Interdisciplinary Philosopher Of Science',
        'Math', 'Science', 'Stats',
        'Accelerating Scientific Computing', 'Drug Discovery',
        'Geographer', 'Gis', 'Philosopher Of Computation',
        'Reasercher', 'Scientific Marketer', 'Ce', 'Pgpds', 'Physician'
    ],
    
    'AI & Machine Learning': [
        'machine learning', 'ml', 'ai', 'deep learning', 'mle', 'artificial intelligence',
        'neural', 'computer vision', 'nlp', 'roboticist', 'autonomous', 'predictive',
        'modeling', 'forecasting',
        '(machine) Learner', 'A.i.m.l Enthusiast', 'Machine Intelligence', 
        'Making Llms', 'Model-based Reinforcement Learning And Robot Learning',
        'Working On Llm-based Simulations',
        'Llm Jockey', 'Part-time Neocortical Rewirer',
        'Replicant Whisperer', 'Tensor Twiddler',
        'Working On Self-driving Cars', 'Ar/vr Eir'
    ],
    
    'Management & Leadership': [
        'manager', 'lead', 'head', 'chief', 'cto', 'ceo', 'director', 'founder',
        'co-founder', 'cofounder', 'owner', 'entrepreneur', 'solopreneur', 'cfo',
        'cio', 'coo', 'cro', 'Gerente De Desarrollo', 'Líder De Ti', 'Managing', 'President',
        'Core Team', 'Former Core Team Member', 'Senior',
        'Staff', 'Team Member', 'Businessman', 'Founding Member, Member', 'Member', 'Sm'
    ],
    
    'Security & DevOps': [
        'security', 'devops', 'sre', 'reliability', 'ops', 'sysadmin', 'admin',
        'infrastructure', 'pentester', 'ciso', 'cybersecurity', 'devsecops',
        'Blue Team-defender', 'Infosec', 'Low Level App Sec', 'Penetration Tester',
        'Penetration Testing', 'Systems'
    ],
    
    'Student & Academic': [
        'student', 'graduate', 'undergraduate', 'major', 'estudante', 'dean',
        'educator', 'teaching', 'academic', 'studying', 'intern',
        'Adjunct', 'Estudiante De Ingeniería De Sistemas',
        'Estudiante De Licenciatura En Ciencias De La Computación',
        'Graduando Em Engenharia Eletrônica E De Computação',
        'Lecturer', 'Masters Candidate', 'Msc Candidate', 'Prof', 
        'Tutor', '商科学生', 'Fresher', 'Sophomore', 'Pre-docente',
        'Aspiring', 'Beginner', 'Expert Beginner', 'Hobbyist', 'Hobbyist / Enthusiast',
        'Learner'
    ],
    
    'Creative & Design': [
        'designer', 'design', 'ui', 'ux', 'artist', 'creative', 'filmmaker',
        'photographer', 'musician', 'tech artist',
        'Composer', 'Visual Communicator', 'Visual Effects Pipeline Supervisor',
        'Creator', 'Músico Y Estudiante'
    ],
    
    'Finance & Trading': [
        'trader', 'trading', 'fintech', 'risk', 'financial', 'crypto',
        'quantitative finance', 'investment', 'economy', 'economist',
        'partner', 'Banker', 'Derivatives', 'Finance', 'Finance Guy', 
        'Fx, Stir And Bond Futures & Options', 'Individual Active Investor',
        'Investor', 'Yield Farmer', '量化研究员', 'Accountant', 'Actuary', 'Bean Counter',
        'Real Estate Lending Expert', 'Smart Contracts', 'Mestrando Em Economia', 'Mfe',
        'Nft', 'Dao'
    ],
    
    'Consulting & Strategy': [
        'consultant', 'consulting', 'strategy', 'advisor', 'strategist',
        'professional', 'specialist', 'Ekonomisk Konsult', 'Innovation Coach',
        'Consultor Sap Fi Jr Certificado Y Analista Programador Cobol Senior',
        'Customer Service', 'General Solutionist'
    ],
    
    'Content & Communication': [
        'writer', 'content', 'technical writer', 'evangelist', 'translator',
        'curator', 'marketing', 'Author', 'Blogger And Gamer', '博主', 'Procurement Officer'
    ],
    'Product & Operations': [
        'product', 'product manager', 'product owner', 'product operations', 
        'product guy', 'project manager', 'project associate', 'program manager',
        'operations', 'orgnzr', 'organizer', 'scrum master', 'agile', 'pm', 'Project Assistant'
    ],
    'Healthcare & Medical': [
        'Family Medicine Resident', 'Folk Healer', 'Healthcare Provider',
        'Midwife', 'Musculoskeletal Imaging', 'Pharmacist',
        'Pharmacist Informatician', 
    ],
    'Legal': [
        'Advogado E Entusiasta De Tecnologias', 'Lawyer', 'Privacy Attorney'
    ],

    'Open Source & Community': [
        'Contributor', 'Gsoc', "Gsoc'22 Contributer", "Gsoc'24",
        "Gsoc'24 Participant", 'Gsoc, Gci Mentor',
        'Open Source Contributor', 'Cncf Ambassador',
        'Full-time Open-sourcerer'
    ],

    'Freelance & Independent': [
        'Freelance', 'Freelancer', 'Full-time Freelancer',
        'Full-time Freelancer, Part-time Open Sourcerer'
    ],
}

def categorize_job(job):
    if pd.isna(job):
        return None
    
    job = job.lower()
    for category, keywords in job_categories.items():
        if any(keyword.lower() in job for keyword in keywords):
            return category
    return 'Other'

# Create new column with job categories
df['JobCategory'] = df['Job'].apply(categorize_job)

# Plot the job categories
category_counts = df['JobCategory'].value_counts().head(10)

fig = go.Figure(data=[
    go.Bar(x=category_counts.index, y=category_counts.values)
])

fig.update_layout(
    title='Stargazers by Job Category',
    xaxis_title='Job Category',
    yaxis_title='Number of Stargazers',
    width=1000,
    height=400
)

fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
# Filter out jobs containing Engineer or Developer, then get top 20
filtered_jobs = df['Job'].str.lower().str.contains('engineer|developer', na=False)
top_job = df[~filtered_jobs]['Job'].value_counts().head(30)

fig = go.Figure(data=[
    go.Bar(x=top_job.index, y=top_job.values)
])

fig.update_layout(
    title='Top Job of Stargazers (Excluding Engineers & Developers)',
    xaxis_title='Job',
    yaxis_title='Number of Stargazers',
    width=1000,
    height=400
)

fig.update_xaxes(tickangle=45)

fig.show()

In [ ]:
def get_word_freq(series):
    # Get English stop words
    stop_words = set(stopwords.words('english'))
    
    # Add any additional common words you want to filter out
    additional_stops = {'i', 'am', 'im', 'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 
                       'to', 'for', 'of', 'with', 'by'}
    stop_words.update(additional_stops)
    
    words = ' '.join(series.dropna()).lower()
    words = re.findall(r'\w+', words)
    
    # Filter out stop words
    words = [word for word in words if word not in stop_words]
    
    return Counter(words)

bio_word_freq = get_word_freq(df['Bio'])
top_words = dict(bio_word_freq.most_common(20))

# Create bar chart using plotly
fig = go.Figure(data=[
    go.Bar(x=list(top_words.keys()), y=list(top_words.values()))
])

fig.update_layout(
    title='Top 20 Words in Stargazer Bios',
    xaxis_title='Word',
    yaxis_title='Frequency',
    width=1000,
    height=400
)

fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
# Get top 30 users by follower count with selected columns
columns_to_show = ['Username', 'Name', 'Location', 'StandardCompany', 'Job', 'Email', 'Twitter', 'Followers']
top_followers = df.nlargest(30, 'Followers')[columns_to_show]

# Format followers number with comma separators
top_followers = top_followers.fillna('')
top_followers['Followers'] = top_followers['Followers'].apply(lambda x: f"{x:,}")

# Display the table using plotly
fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(columns_to_show),
        fill_color='paleturquoise',
        align='left',
        font=dict(size=12)
    ),
    cells=dict(
        values=[top_followers[col] for col in columns_to_show],
        fill_color='lavender',
        align='left',
        font=dict(size=11)
    )
)])

fig.update_layout(
    title='Top 30 Stargazers by Follower Count',
    width=1200,
    height=800
)

fig.show()

In [ ]:
!jupyter nbconvert --to html --no-input stargazers_analysis.ipynb